In [4]:
import json
import os 

file = "results/__home__gstmchen__optimized-model-merging__generative__outs__llama_merged__frank_wolfe__frank_wolfe_merge2_01/results_2025-02-07T11-34-16.745375.json"
tie_prefix = "__home__gstmchen__optimized-model-merging__generative__outs__llama_merged__frank_wolfe__frank_wolfe_merge_"
task_arith_prefix = "__home__gstmchen__optimized-model-merging__generative__outs__llama_merged__frank_wolfe__frank_wolfe_merge2_"

baseline_scores = {
    'cnn_dailymail': 0.12675733665329772, 
    'mmlu': 0.4272570442783209, 
    'piqa': 0.758, 
    'wmt16-ro-en': 18.599623556677553}

tie_dict = {
    'cnn_dailymail': [], 
    'mmlu': [],
    'piqa': [],
    'wmt16-ro-en': [],
}
task_arith_dict = {
    'cnn_dailymail': [],
    'mmlu': [],
    'piqa': [],
    'wmt16-ro-en': [],
}

metric_map = {
    'cnn_dailymail': 'rouge,none',
    'mmlu': 'acc,none',
    'piqa': 'acc,none',
    'wmt16-ro-en': 'bleu,none',
}

for folder_name in os.listdir("results"):
    file_name = sorted(f for f in os.listdir("results/" + folder_name) if f.startswith("results_"))[-1]
    with open("results/" + folder_name + "/" + file_name, 'r') as f:
        results = json.load(f)['results']
    for key in results.keys():
        if key not in metric_map:
            continue
        d = results[key][metric_map[key]] / baseline_scores[key]
        if folder_name.startswith(tie_prefix):
            tie_dict[key].append(d)
        elif folder_name.startswith(task_arith_prefix):
            task_arith_dict[key].append(d)

print(tie_dict)
print(task_arith_dict)

max_tie_score = 0 
max_tie_key = None
max_task_arith_score = 0
max_task_arith_key = None
task_num = len(tie_dict['cnn_dailymail'])
for i in range(task_num):
    tie_score = sum(tie_dict[key][i] for key in tie_dict.keys()) / task_num
    task_arith_score = sum(task_arith_dict[key][i] for key in task_arith_dict.keys()) / task_num
    if tie_score > max_tie_score:
        max_tie_score = tie_score
        max_tie_key = i
    if task_arith_score > max_task_arith_score:
        max_task_arith_score = task_arith_score
        max_task_arith_key = i

print(max_tie_score, max_tie_key)
print(max_task_arith_score, max_task_arith_key)

{'cnn_dailymail': [0.43180575314268377, 0.43158987392961773, 0.4165613655782313, 0.4155405794753616], 'mmlu': [1.0067294751009421, 1.003460872909056, 1.003460872909056, 1.0032686021918862], 'piqa': [1.0343007915567282, 1.0343007915567282, 1.0343007915567282, 1.0343007915567282], 'wmt16-ro-en': [1.5152866771017584, 1.513853890711738, 1.5141694691005363, 1.5130880444042472]}
{'cnn_dailymail': [0.4335744366634509, 0.43002517920858696, 0.4335744366634509, 0.421943356621873], 'mmlu': [1.004999038646414, 1.0051913093635838, 1.004999038646414, 1.0030763314747162], 'piqa': [1.0343007915567282, 1.0343007915567282, 1.0343007915567282, 1.0343007915567282], 'wmt16-ro-en': [1.5148598674999743, 1.5177421423673756, 1.5148598674999743, 1.5133908796773319]}
0.9970306742255282 0
0.9969335335916418 0


In [1]:
import json
import os 

file = "results_baseline/cnn/__home__gstmchen__optimized-model-merging__llama__llama-cnn__merged/results_2025-02-07T08-21-24.606050.json"

baseline_dict = {
    'cnn_dailymail': [],
    'mmlu': [],
    'piqa': [],
    'wmt16-ro-en': [],
}

metric_map = {
    'cnn_dailymail': 'rouge,none',
    'mmlu': 'acc,none',
    'piqa': 'acc,none',
    'wmt16-ro-en': 'bleu,none',
}

for folder_name1 in os.listdir("results_baseline"):
  for folder_name2 in os.listdir("results_baseline/" + folder_name1):
    with open(f"results_baseline/{folder_name1}/{folder_name2}/{sorted(f for f in os.listdir(f'results_baseline/{folder_name1}/{folder_name2}') if f.startswith('results_'))[-1]}", 'r') as f:
        results = json.load(f)['results']
    for key, value in results.items():
        if key in metric_map:
            baseline_dict[key].append(value[metric_map[key]])

print(baseline_dict)

{'cnn_dailymail': [0.12675733665329772], 'mmlu': [0.4272570442783209], 'piqa': [0.758], 'wmt16-ro-en': [18.599623556677553]}


In [1]:
import os 
os.environ['CUDA_VISIBLE_DEVICES'] = '2'
from vllm import LLM 

model = LLM(
  model='../optimized-model-merging/llama/llama-mmlu/merged', 
  tokenizer='meta-llama/Llama-2-7b-hf',
  gpu_memory_utilization=0.5
)

/home/gstmchen/miniconda3/envs/lm-eval/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


INFO 02-07 00:33:45 __init__.py:183] Automatically detected platform cuda.


2025-02-07 00:33:45,799	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


INFO 02-07 00:33:46 config.py:2364] Downcasting torch.float32 to torch.float16.
INFO 02-07 00:33:54 config.py:526] This model supports multiple tasks: {'generate', 'classify', 'score', 'reward', 'embed'}. Defaulting to 'generate'.
INFO 02-07 00:33:54 llm_engine.py:232] Initializing a V0 LLM engine (v0.7.1) with config: model='../optimized-model-merging/llama/llama-mmlu/merged', speculative_config=None, tokenizer='meta-llama/Llama-2-7b-hf', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.float16, max_seq_len=4096, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto,  device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='xgrammar'), observability_config=ObservabilityConfig(otlp_traces_endpoint=None, collect_model_forward_time

[W207 00:33:56.412516038 socket.cpp:752] [c10d] The client socket cannot be initialized to connect to [ws5]:53411 (errno: 97 - Address family not supported by protocol).
Loading safetensors checkpoint shards:   0% Completed | 0/6 [00:00<?, ?it/s]
Loading safetensors checkpoint shards:  17% Completed | 1/6 [00:01<00:05,  1.08s/it]
Loading safetensors checkpoint shards:  33% Completed | 2/6 [00:02<00:04,  1.25s/it]
Loading safetensors checkpoint shards:  50% Completed | 3/6 [00:03<00:03,  1.29s/it]
Loading safetensors checkpoint shards:  67% Completed | 4/6 [00:05<00:02,  1.33s/it]
Loading safetensors checkpoint shards:  83% Completed | 5/6 [00:06<00:01,  1.35s/it]
Loading safetensors checkpoint shards: 100% Completed | 6/6 [00:07<00:00,  1.21s/it]
Loading safetensors checkpoint shards: 100% Completed | 6/6 [00:07<00:00,  1.25s/it]



INFO 02-07 00:34:05 model_runner.py:1116] Loading model weights took 12.5523 GB
INFO 02-07 00:34:07 worker.py:266] Memory profiling takes 1.72 seconds
INFO 02-07 00:34:07 worker.py:266] the current vLLM instance can use total_gpu_memory (39.50GiB) x gpu_memory_utilization (0.50) = 19.75GiB
INFO 02-07 00:34:07 worker.py:266] model weights take 12.55GiB; non_torch_memory takes 0.09GiB; PyTorch activation peak memory takes 0.35GiB; the rest of the memory reserved for KV Cache is 6.75GiB.
INFO 02-07 00:34:07 executor_base.py:108] # CUDA blocks: 863, # CPU blocks: 512
INFO 02-07 00:34:07 executor_base.py:113] Maximum concurrency for 4096 tokens per request: 3.37x
INFO 02-07 00:34:11 model_runner.py:1435] Capturing cudagraphs for decoding. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI. If out-of-memory error occurs during cudagraph capture, consider decreasing `gpu_memory_utili

Capturing CUDA graph shapes: 100%|██████████| 35/35 [00:23<00:00,  1.46it/s]

INFO 02-07 00:34:35 model_runner.py:1563] Graph capturing finished in 24 secs, took 0.24 GiB
INFO 02-07 00:34:35 llm_engine.py:429] init engine (profile, create kv cache, warmup model) took 30.27 seconds
